# **InfoShape Experiments** 
#### Bill Wu


### All experiments for our InfoShape [paper](https://arxiv.org/abs/2210.15034) are included here.

---------------------------------------

Abstract:

_The use of mutual information as a tool in private data sharing has remained an open challenge due to the difficulty of its estimation in practice. In this paper, we propose InfoShape, a task-based encoder that aims to remove unnecessary sensitive information from training data while maintaining enough relevant information for a particular ML training task. We achieve this goal by utilizing mutual information estimators that are based on neural networks, in order to measure two performance metrics, privacy and utility. Using these together in a Lagrangian optimization, we train a separate neural network as a lossy encoder. We empirically show that InfoShape is capable of shaping the encoded samples to be informative for a specific downstream task while eliminating unnecessary sensitive information. Moreover, we demonstrate that the classification accuracy of downstream models has a meaningful connection with our utility and privacy measures._

---------------------------------------

Current Infoshape [repository](https://github.com/billywu1029/infoshape).

Original [thesis work](https://github.com/billywu1029/mine-pytorch).

[MINE Paper](https://arxiv.org/pdf/1801.04062.pdf)

- Original [MINE repository](https://github.com/gtegner/mine-pytorch), thanks to [Gustaf Tegner](https://github.com/gtegner).

[ReMINE Paper](https://openreview.net/forum?id=Lvb2BKqL49a)

Theoretical Materials on MI Variational Bounds [paper](https://arxiv.org/pdf/1905.06922.pdf) (potential future direction).

- Their google [colab notebook](https://colab.research.google.com/github/google-research/google-research/blob/master/vbmi/vbmi_demo.ipynb).


### High Level Organization
1. Code Setup (MINE library, constants, classifier training procedure, evaluation criteria, model setup)
2. Synthetic Data Setup
3. Experiments
  - Classifier performance on original unencoded data (both public and sensitive features)
  - Calculate Entropy of both the public and sensitive labelling functions, H(L(X)) and H(S(X)), respectively (Figure 3 upper bounds in the paper).
  - Classifier performance on data transformed by an untrained encoder (both public and sensitive labels)
  - Use InfoShape to train the encoder via dual optimization of utility score and privacy leakage.
  - Reevaluate classifier performance on data transformed by the _trained_ encoder (both public and sensitive labels)
  - Compare classification results to hypothetical baseline of adding Gaussian noise to input data (mimicking simple Differential Privacy)

# **1. Code Setup**

### Imports

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mine.datasets import MultivariateNormalDataset
import json

!pip install pytorch_lightning
import pytorch_lightning

from tqdm import tqdm
from pytorch_lightning import Trainer, seed_everything

import torch.nn as nn
from torch.nn import functional as F

import time
import logging
logging.getLogger().setLevel(logging.ERROR)

from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler

from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from mine import MutualInformationEstimator
from mine import Mine

### Set GPU Device and Set Seed

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_gpus = 1 if device=='cuda' else 0
print(device)

seed = 1
seed_everything(seed, workers=True)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Numpy RNG
np.random.seed(seed)

### Constants

In [ ]:
SAVE_PATH = "data/"  # Location for saved classification results
ENC_SAVE_PATH = "model/"  # Location for saving encoder weights per training epoch
BETA = 1  # Used for the Lagrangian optimization in the loss term for InfoShape, ie I(Z; L(X)) - BETA * I(Z; S(X))
MINE_BATCH_SIZE = 2000  # batch size for loading dataset into MINE
N_ENC_OUT_NODES = 3  # Number of output nodes in InfoShape's encoder's output layer
N_CLASSIFIER_TRAINING_EPOCHS = 50

### Classifier and Encoder Setup

In [ ]:
class DenseClassifier(nn.Module):
    def __init__(self, in_nodes, hidden_nodes=20):
        super(DenseClassifier, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_nodes, hidden_nodes), 
            nn.ReLU(), 
            nn.Linear(hidden_nodes, hidden_nodes),
            nn.ReLU(),
            nn.Linear(hidden_nodes, 1), 
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.to(device)
        return self.main(x)

    def train_classifier(self, train_loader, epochs=N_CLASSIFIER_TRAINING_EPOCHS):
        optimizer = torch.optim.SGD(self.parameters(), lr=1e-4, momentum=0.9)
        for epoch in tqdm(range(epochs)):
            self.train()
            train_loss = 0
            for x, y in train_loader:
                x, y = x.to(device).float(), y.to(device).float()
                y_hat = self(x).squeeze()
                loss = F.binary_cross_entropy(y_hat, y, reduction="sum")
                train_loss += loss.item()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # Average loss per image
            # If avg loss per pixel, divide again by inp_dim * inp_dim
            avg_train_loss = train_loss / len(train_loader.dataset)
            print(f'====> Epoch: {epoch} Average loss: {avg_train_loss:.4f}')

    def evaluate(self, test_loader, experiment_save_file=None):
        self.eval()
        test_data, test_labels = test_loader.dataset.data.float(), test_loader.dataset.targets.float()
        preds = self(test_data).squeeze()
        y_true = test_labels.detach().cpu().numpy()
        y_score = preds.detach().cpu().numpy()

        fpr, tpr, thresholds = roc_curve(y_true, y_score)
        auc = roc_auc_score(y_true, y_score)
        print(f"AUC: {auc:.4f}")

        test_loss = F.binary_cross_entropy(preds, test_labels.to(device))
        print(test_loss.shape, test_loss)
        print(f"====> Test loss: {test_loss:.4f}")

        if experiment_save_file is not None:
            with open(SAVE_PATH + experiment_save_file, "w") as f:
                json.dump({"fpr": fpr.tolist(), "tpr": tpr.tolist(), "thresholds": thresholds.tolist()}, f)

        plt.plot(fpr, tpr, marker='.')
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate' )
        plt.show()

class DenseEncoder(nn.Module):
    def __init__(self, in_dim, hidden_nodes=10, out_nodes=N_ENC_OUT_NODES):
        super(DenseEncoder, self).__init__()
        in_nodes = in_dim[0]
        self.out_nodes = out_nodes
        self.main = nn.Sequential(
            # Want 2 layers for more nonlinearity in the encoded data
            nn.Linear(in_nodes, hidden_nodes),
            nn.Tanh(),
            nn.Linear(hidden_nodes, out_nodes),
            nn.Tanh(),
        )

    def forward(self, x):
        x = x.to(device)
        return self.main(x)

# **2. Dataset Setup**

In [ ]:
class SyntheticDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y
    
    def __len__(self):
        return len(self.data)

In [ ]:
N_SAMPLES = 10000
BATCH_SIZE = 100
N_FEATURES = 10

# Used to apply Gaussian noise on inputs X to show that both ROCs of public and
# sensitive label classification go down towards our ROC on classification on Z
ADD_GAUSSIAN_NOISE = True  # Set to True if testing noise experiment
MU = 0
VARIANCE = 1
STDDEV = VARIANCE ** 0.5

# defaults: n_informative=2, n_redundant=2, n_classes=2, shuffle=True
# -> 2 primary features, 2 linear combinations of them, and the rest are random noise
# binary labels for each feature
X, y = make_classification(n_samples=N_SAMPLES, n_features=N_FEATURES, n_informative=3, n_classes=4, random_state=seed, class_sep=1)

if ADD_GAUSSIAN_NOISE:
    noise_matrix = np.zeros_like(X)
    for i in range(N_SAMPLES):
        # Done for each sample -> gaussians are independent
        noise_matrix[i] = np.random.normal(MU, STDDEV, N_FEATURES)
        if i == 0 or i == N_SAMPLES - 1:
            print(noise_matrix[i])
    X += noise_matrix

# X: (N_SAMPLES, N_FEATURES), Y: (N_SAMPLES, )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Public label set
y_train_pub = np.where(y_train // 2 == 0, np.ones(y_train.shape), np.zeros_like(y_train))
y_train_pri = np.where(y_train % 2 == 0, np.ones(y_train.shape), np.zeros_like(y_train))
y_test_pub = np.where(y_test // 2 == 0, np.ones(y_test.shape), np.zeros_like(y_test))
y_test_pri = np.where(y_test % 2 == 0, np.ones(y_test.shape), np.zeros_like(y_test))

train_dataset_pub = SyntheticDataset(torch.from_numpy(X_train), torch.from_numpy(y_train_pub))
test_dataset_pub = SyntheticDataset(torch.from_numpy(X_test), torch.from_numpy(y_test_pub))
train_dataset_pri = SyntheticDataset(torch.from_numpy(X_train), torch.from_numpy(y_train_pri))
test_dataset_pri = SyntheticDataset(torch.from_numpy(X_test), torch.from_numpy(y_test_pri))

train_loader_pub = DataLoader(train_dataset_pub, batch_size=BATCH_SIZE, shuffle=True)
test_loader_pub = DataLoader(test_dataset_pub, batch_size=BATCH_SIZE, shuffle=True)
train_loader_pri = DataLoader(train_dataset_pri, batch_size=BATCH_SIZE, shuffle=True)
test_loader_pri = DataLoader(test_dataset_pri, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
y_train_pub[:10]

In [ ]:
y_train_pri[:10]

### Visualize Data Points

Public Labels

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(train_dataset_pub.data[:,0], train_dataset_pub.data[:,1], train_dataset_pub.data[:,2], c=train_dataset_pub.targets)

Private Labels

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(train_dataset_pri.data[:,0], train_dataset_pri.data[:,1], train_dataset_pri.data[:,2], c=train_dataset_pri.targets)

# **3. Experiments**

### Initial Classification
This establishes a sample baseline for what both an honest researcher for a downstream AI task as well a malicious adversary could accomplish. 

##### Public Labels

In [ ]:
# Using a very simple dense classifier for now as a proof of concept. 
model = DenseClassifier(N_FEATURES).to(device)
model

In [ ]:
model.train_classifier(train_loader_pub, epochs=N_CLASSIFIER_TRAINING_EPOCHS)

In [ ]:
EXP_SAVE_FILE = "add noise og data roc auc.json" if ADD_GAUSSIAN_NOISE else "og data roc auc.json"
model.evaluate(test_loader_pub, experiment_save_file=EXP_SAVE_FILE)

##### Private Labels

In [ ]:
# Reinitialize the classifier
model = DenseClassifier(N_FEATURES).to(device)
model

In [ ]:
model.train_classifier(train_loader_pri, epochs=N_CLASSIFIER_TRAINING_EPOCHS)

In [ ]:
EXP_SAVE_FILE_PRI = "add noise og roc auc pri.json" if ADD_GAUSSIAN_NOISE else "og roc auc pri.json"
model.evaluate(test_loader_pri, experiment_save_file=EXP_SAVE_FILE_PRI)

### [INFOSHAPE SETUP] Dual Optimization Procedure

Core InfoShape contribution: putting everything together to get ReMINE estimates for I(Z; L(X)) and I(Z; S(X)) to form the dual objective function to train a 2-layer dense encoder.

In [ ]:
class MINE(nn.Module):
    def __init__(self, enc_out_num_nodes):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(enc_out_num_nodes + 1, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 1)
        )

    def forward(self, z, labels):
        z, labels = z.float().to(device), labels.float().to(device)
        z = z.view(z.size(0), -1)
        cat = torch.cat((z, labels.unsqueeze(-1)), 1)
        return self.layers(cat)

class DualOptimizationDenseEncoder(nn.Module):
    def __init__(self, data_loader, mine_epochs_privacy, mine_epochs_utility, enc_out_nodes=N_ENC_OUT_NODES, beta=BETA, enc_shape=N_FEATURES, private_labels=None):
        super().__init__()
        self.encoder = DenseEncoder((enc_shape,), out_nodes=enc_out_nodes).to(device)
        self.data_loader = data_loader
        self.private_labels = torch.from_numpy(private_labels)  # Fail fast with None value if misconfigured
        self.mine_epochs_privacy = mine_epochs_privacy
        self.mine_epochs_utility = mine_epochs_utility
        self.beta = beta

    def get_MINE(self, transformed_data_loader, enc_out_num_nodes, mine_epochs, train_epoch, K=MINE_BATCH_SIZE, gradient_batch_size=1, func_str=None):
        stats_network = MINE(enc_out_num_nodes).to(device)
        mi_estimator = Mine(stats_network, loss='mine').to(device)
        func_str = f"training epoch={train_epoch}: f(x)=DenseEnc(x) {enc_out_num_nodes} nodes" if not func_str else func_str

        kwargs = {
            'mine': mi_estimator,
            'lr': 1e-4,
            'batch_size': K,
            'alpha': 0.1,  # Used as the ema weight in MINE
            'func': func_str,
            'train_loader': transformed_data_loader,
            # Determines how many minibatches (MINE iters) of gradients get accumulated before optimizer step gets applied
            # Meant to stabilize the MINE curve for [hopefully] better encoder training performance
            'gradient_batch_size': gradient_batch_size
        }

        logger = TensorBoardLogger(
            "lightning_logs",
            name=f"{EXPERIMENT} BS={K}",
            version=f"{func_str}, BS={K}"
        )

        model = MutualInformationEstimator(loss='mine', **kwargs).to(device)
        return model, logger

    def forward(self, epoch, num_batches_final_MI, include_privacy=True, include_utility=True, K=MINE_BATCH_SIZE, gradient_batch_size=1):
        # Get encoder transformed data
        transformedimgs = self.encoder(self.data_loader.dataset.data.float())

        labels_public = self.data_loader.dataset.targets.float()
        labels_private = self.private_labels.float()

        z_train_utility_detached = SyntheticDataset(transformedimgs.detach(), labels_public.detach())
        z_train_privacy_detached = SyntheticDataset(transformedimgs.detach(), labels_private.detach())
        z_train_loader_utility_detached = DataLoader(z_train_utility_detached, K, shuffle=True)
        z_train_loader_privacy_detached = DataLoader(z_train_privacy_detached, K, shuffle=True)

        # Get MINE model (sitting in Pytorch lightning module)
        model_MINE_utility, logger_utility = self.get_MINE(
            z_train_loader_utility_detached, self.encoder.out_nodes, self.mine_epochs_utility, epoch, K=K, gradient_batch_size=gradient_batch_size)
        model_MINE_privacy, logger_privacy = self.get_MINE(
            z_train_loader_privacy_detached, self.encoder.out_nodes, self.mine_epochs_privacy, epoch, K=K, gradient_batch_size=gradient_batch_size)

        # Optimize MINE estimate, "train" MINE
        last_mi_utility = last_mi_privacy = 0
        if include_utility:
            trainer_utility = Trainer(max_epochs=self.mine_epochs_utility, logger=logger_utility, gpus=1)
            trainer_utility.fit(model_MINE_utility)

            ## -------- Calculate I(T(x); L(x)) estimate after MINE training ---------- ##
            # **IMPORTANT**: Use the non-detached og transformedimgs so that gradients are retained
            z_train_utility = SyntheticDataset(transformedimgs, labels_public.float())
            z_train_loader_utility = DataLoader(z_train_utility, K, shuffle=True)
            model_MINE_utility.energy_loss.to(device)
            sum_MI_utility = 0

            # Average MI across num_batches_final_MI batches to lower variance
            # Batches are K random samples from the dataset after all
            assert num_batches_final_MI < len(z_train_loader_utility.dataset) / K
            utility_it = iter(z_train_loader_utility)
            for i in range(num_batches_final_MI):
                Tx, Lx = next(utility_it)
                Tx.to(device)
                Lx.to(device)
                sum_MI_utility += model_MINE_utility.energy_loss(Tx, Lx)
                
            # MINE loss = -1 * MI estimate since we are maximizing using gradient descent still
            last_mi_utility = -1 * sum_MI_utility / num_batches_final_MI

        if include_privacy:
            trainer = Trainer(max_epochs=self.mine_epochs_privacy, logger=logger_privacy, gpus=1)
            trainer.fit(model_MINE_privacy)

            ## -------- Calculate I(T(x); S(x)) estimate after MINE training ---------- ##
            z_train_privacy = SyntheticDataset(transformedimgs, labels_private)
            z_train_loader_privacy = DataLoader(z_train_privacy, K, shuffle=True)
            model_MINE_privacy.energy_loss.to(device)

            assert num_batches_final_MI < len(z_train_loader_privacy.dataset) / K
            sum_MI_privacy = 0
            privacy_it = iter(z_train_loader_privacy)
            prev_mi = None
            for i in range(num_batches_final_MI):
                Tx, Sx = next(privacy_it)
                Tx.to(device)
                Sx.to(device)
                sum_MI_privacy += model_MINE_privacy.energy_loss(Tx, Sx)
                
            last_mi_privacy = -1 * sum_MI_privacy / num_batches_final_MI

        print(f"final MI values: utility: {last_mi_utility}, privacy: {last_mi_privacy}")
        return last_mi_utility, last_mi_privacy

    def train_encoder(
        self, 
        num_enc_epochs=10, 
        num_batches_final_MI=100, 
        save_enc_weights=False, 
        include_privacy=True, 
        include_utility=True,
        K=MINE_BATCH_SIZE,
        gradient_batch_size=1,
        enc_save_path=ENC_SAVE_PATH,
    ):
        # Encoder's training params
        learning_rate = 1e-3
        encoder_optimizer = torch.optim.Adam(
            self.encoder.parameters(),
            lr=learning_rate,
        )
        self.encoder.train()

        for epoch in range(num_enc_epochs):
            mi_utility, mi_privacy = self.forward(  # TODO what are these values, are they -utiilty and -privacy? Make sure 
                epoch, num_batches_final_MI, include_privacy=include_privacy, include_utility=include_utility, K=K, gradient_batch_size=gradient_batch_size
            )
            encoder_optimizer.zero_grad()            
            loss = -mi_utility + self.beta * mi_privacy
            loss.backward()
            encoder_optimizer.step()

            if save_enc_weights:
                # Don't save the state dict since that doesn't include the model parameters + their gradients
                # Options were to save entire model or optimizer's state dict:
                # https://discuss.pytorch.org/t/how-to-save-the-requires-grad-state-of-the-weights/52906/6
                print(f"Saving weights to {enc_save_path}")
                torch.save(self.encoder, enc_save_path + f"{EXPERIMENT} epoch={epoch}.pt")
                torch.save(encoder_optimizer.state_dict(), enc_save_path + f"[optimizer] {EXPERIMENT} epoch={epoch}.pt")

            print(f'====> Epoch: {epoch} Utility MI I(T(x); L(x)): {mi_utility:.8f}')
            print(f'====> Epoch: {epoch} Privacy MI I(T(x); S(x)): {mi_privacy:.8f}')
            print(f'====> Epoch: {epoch} Loss: {loss:.8f}')

### Entropy Baselines
Use ReMINE to empirically converge to a lower bound for different mutual information quantiies representing:

TODO: Check?
1. Entropy of the dataset H(X) = I(X; X)
2. Entropy of the labelling function H(L(X)) >= I(X; L(X)) = H(L(X)) - H(L(X)|X)
3. Entropy of the private labelling function H(S(X)) >= I(X; S(X)) = H(S(X)) - H(S(X)|X)

#### Calculate H(X) entropy of the generated data
TODO: remove??

In [ ]:
EXPERIMENT = f"[SYNTHETIC DATA | PUBLIC PRIVATE LABELS | H(X) COMPUTE] REMINE BS=2K C=0 λ=0.1 MIb=10"

t = MINE(N_FEATURES, N_FEATURES).to(device)
mi_estimator = Mine(t, loss='mine').to(device)
func_str = f"I(x;x) {N_FEATURES} nodes"
lr = 1e-4

train_dataset_HX = SyntheticDataset(torch.from_numpy(X_train), torch.from_numpy(X_train))
train_loader_HX = DataLoader(train_dataset_HX, batch_size=BATCH_SIZE, shuffle=True)

kwargs = {
    'mine': mi_estimator,
    'lr': lr,
    'batch_size': MINE_BATCH_SIZE,
    'alpha': 0.1,
    'func': func_str,
    'train_loader': train_loader_HX,  # TODO: Repeat for private labels dataset too: train_loader_pri
    # Determines how many minibatches (MINE iters) of gradients get accumulated before optimizer step gets applied
    # Meant to stabilize the MINE curve for better encoder training performance
    'gradient_batch_size': 10
}

logger = TensorBoardLogger(
    "lightning_logs",
    name=EXPERIMENT",
    version=f"{func_str}, BS: {MINE_BATCH_SIZE}"
)

model = MutualInformationEstimator(loss='mine', **kwargs).to(device)

trainer = Trainer(max_epochs=1500, logger=logger, gpus=1)
trainer.fit(model)

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [ ]:
%tensorboard --logdir="lightning_logs/[SYNTHETIC DATA | PUBLIC PRIVATE LABELS | H(X) COMPUTE] REMINE BS=2K C=0 λ=0.1 MIb=10 privacy BS=2000" --max_reload_threads 4 --samples_per_plugin scalars=1000

#### Calculate H(L(X)) entropy of the public labeling function

In [ ]:
EXPERIMENT = f"[SYNTHETIC DATA | PUBLIC PRIVATE LABELS | H(L(X)) PUB EST] REMINE BS=2K C=0 λ=0.1 10 final batches MI"
train_loader_pub_HLx = DataLoader(train_dataset_pub, batch_size=MINE_BATCH_SIZE, shuffle=True)

t = MINE(N_FEATURES).to(device)
mi_estimator = Mine(t, loss='mine').to(device)
func_str = f"I(x;L(x)) {N_FEATURES} nodes"
lr = 1e-4

kwargs = {
    'mine': mi_estimator,
    'lr': lr,
    'batch_size': MINE_BATCH_SIZE,
    'alpha': 0.1,
    'func': func_str,
    'train_loader': train_loader_pub_HLx,
    # Determines how many minibatches (MINE iters) of gradients get accumulated before optimizer step gets applied
    # Meant to stabilize the MINE curve for better encoder training performance
    'gradient_batch_size': 10
}

logger = TensorBoardLogger(
    "lightning_logs",
    name=f"{EXPERIMENT} utility BS={MINE_BATCH_SIZE}",
    version=f"{func_str}, BS: {MINE_BATCH_SIZE}"
)

model = MutualInformationEstimator(loss='mine', **kwargs).to(device)

trainer = Trainer(max_epochs=5000, logger=logger, gpus=1)
trainer.fit(model)

In [ ]:
# Upload these to tensorboard to extract numerical values directly to a pandas df
!tensorboard dev upload --logdir "lightning_logs/[SYNTHETIC DATA | PUBLIC PRIVATE LABELS | H(L(X)) PUB EST] REMINE BS=2K C=0 λ=0.1 10 final batches MI utility BS=2000" \
  --name "ReMINE [PUBLIC PRIVATE LABELS | H(L(X))] REMINE BS=2K utility" \
  --one_shot

#### Calculate H(S(X)) entropy of the labeling function (Private labels)

In [ ]:
EXPERIMENT = f"[SYNTHETIC DATA | PUBLIC PRIVATE LABELS | H(S(X)) PRI EST] REMINE BS=2K C=0 λ=0.1 10 final batches MI"
train_loader_pri_HSx = DataLoader(train_dataset_pri, batch_size=MINE_BATCH_SIZE, shuffle=True)

lr = 1e-4
t = MINE(N_FEATURES).to(device)
mi_estimator = Mine(t, loss='mine').to(device)
func_str = f"I(x;S(x)) {N_FEATURES} nodes"

kwargs = {
    'mine': mi_estimator,
    'lr': lr,
    'batch_size': MINE_BATCH_SIZE,
    'alpha': 0.1,
    'func': func_str,
    'train_loader': train_loader_pri_HSx,
    # Determines how many minibatches (MINE iters) of gradients get accumulated before optimizer step gets applied
    # Meant to stabilize the MINE curve for better encoder training performance
    'gradient_batch_size': 10
}

logger = TensorBoardLogger(
    "lightning_logs",
    name=f"{EXPERIMENT} utility BS={MINE_BATCH_SIZE}",
    version=f"{func_str}, BS={MINE_BATCH_SIZE}"
)

model = MutualInformationEstimator(loss='mine', **kwargs).to(device)

trainer = Trainer(max_epochs=5000, logger=logger, gpus=1)
trainer.fit(model)

In [ ]:
!tensorboard dev upload --logdir "lightning_logs/[SYNTHETIC DATA | PUBLIC PRIVATE LABELS | H(S(X)) PRI EST] REMINE BS=2K C=0 λ=0.1 10 final batches MI utility BS=2000" \
  --name "ReMINE [PUBLIC PRIVATE LABELS | H(S(X))] REMINE BS=2K utility" \
  --one_shot

#### Extract tensorboard values to Pandas DF

In [ ]:
import pandas as pd
import tensorboard as tb
from packaging import version

major_ver, minor_ver, _ = version.parse(tb.__version__).release
assert major_ver >= 2 and minor_ver >= 3, \
    "This notebook requires TensorBoard 2.3 or later."
print("TensorBoard version: ", tb.__version__)

In [ ]:
experiment_id = "T6QwUgmRRFmvlGNmCPV9pw"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
HLx_df = experiment.get_scalars()
HLx_df

In [ ]:
experiment_id = "FeN5wfEpTQyK8ZCeG8WubQ"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
HSx_df = experiment.get_scalars()
HSx_df

In [ ]:
import json

with open("data/icassp/privacy/remine_HLx.json", "w") as f:
    f.write(HLx_df.to_json(orient="records", lines=True))
with open("data/icassp/utility/remine_HSx.json", "w") as g:
    g.write(HSx_df.to_json(orient="records", lines=True))

### Classification on Encoded Data (Untrained Encoder)
Using a simple untrained encoder (DenseEncoder is just 2 dense layers with nonlinearity), establish a baseline for classification performance on both public and private labels. This is the same as the "Initial Classification" section, except with encoded data.

##### Public Labels

In [ ]:
enc = DenseEncoder((N_FEATURES,), out_nodes=N_ENC_OUT_NODES).to(device)
train_transform_pub = enc(train_dataset_pub.data.float()).detach()
test_transform_pub = enc(test_dataset_pub.data.float()).detach()
train_data_transform_pub = SyntheticDataset(
    train_transform_pub,
    train_dataset_pub.targets
)
test_data_transform_pub = SyntheticDataset(
    test_transform_pub,
    test_dataset_pub.targets
)

train_loader_transform_pub = DataLoader(train_data_transform_pub, batch_size=BATCH_SIZE, shuffle=True)
test_loader_transform_pub = DataLoader(test_data_transform_pub, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model_enctrans = DenseClassifier(enc.out_nodes).to(device)
model_enctrans

In [ ]:
model_enctrans.train_classifier(train_loader_transform_pub, epochs=N_CLASSIFIER_TRAINING_EPOCHS)

In [ ]:
model_enctrans.evaluate(test_loader_transform_pub, experiment_save_file="untrainedenc data roc auc pub.json")

##### Private Labels

In [ ]:
# Use the same enc as above, just to demonstrate a point (we'll be using the same trained encoder after enc training w ReMINE)
# enc = DenseEncoder((N_FEATURES,), out_nodes=N_ENC_OUT_NODES).to(device)
train_transform_pri = enc(train_dataset_pri.data.float()).detach()
test_transform_pri = enc(test_dataset_pri.data.float()).detach()
train_data_transform_pri = SyntheticDataset(
    train_transform_pri,
    train_dataset_pri.targets
)
test_data_transform_pri = SyntheticDataset(
    test_transform_pri,
    test_dataset_pri.targets
)

train_loader_transform_pri = DataLoader(train_data_transform_pri, batch_size=BATCH_SIZE, shuffle=True)
test_loader_transform_pri = DataLoader(test_data_transform_pri, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model_enctrans = DenseClassifier(enc.out_nodes).to(device)
model_enctrans

In [ ]:
model_enctrans.train_classifier(train_loader_transform_pri, epochs=N_CLASSIFIER_TRAINING_EPOCHS)

In [ ]:
model_enctrans.evaluate(test_loader_transform_pri, experiment_save_file="untrainedenc data roc auc pri.json")

### **[INFOSHAPE]** Training Procedure for Encoder

Utility Score: the negative of the average uncertainty about the public label given its encoded representation.
\begin{equation}
    M_\text{utility}(T)\triangleq-\mathbf{H}[L(x)|T(x)]
\end{equation}

Privacy Leakage: the average uncertainty about the private label given its encoded representation.
\begin{equation}
    M_\text{privacy}(T)\triangleq\mathbf{H}[S(x)|T(x)]
\end{equation}

Train an encoder to minimize the dual optimization objective (where $T^*$ is the "optimal" encoder): 
\begin{align}
    T^*& = \arg\min_{T\in\mathcal{T}}  M_\text{utility}(T)+\beta M_\text{privacy}(T)\\
    &= \arg\min_{T\in\mathcal{T}} I(T(x); L(x)) + \beta I(T(x); S(x))
\end{align}

Please see Equation 5 and Algorithm 1 in our [paper](https://arxiv.org/abs/2210.15034) for derivations.

-----------------------------------

The code for this section is quite convoluted with many stack frames and Pytorch Lightning calls that abstract away the logic. The high level organization is as follows:
1. Initialize the DualOptimizationDenseEncoder class with params and data
2. Call train_encoder() and indicate that we want to save the encoder weights 
3. For each epoch, do:
  - Use the encoder to transform a copy of the dataset
  - Create and _detach from the computation graph_ (very important!) an intermediate dataset to feed into the ReMINE statistics network to calculate utility score, ie I(Z; L(X)), where L is the public labelling function.
  - Set up the ReMINE statistics network and MutualInformationEstimator for calculating the utility score (lower bound on I(Z; L(X))) and then train it to maximize the lower bound.
  - Create an intermediate dataset (_not detached this time!_) and use the optimized ReMINE network to calculate an average utility score across `num_batches_final_MI` iterations.
  - Repeat each of the steps in Step 3 so far for private labels and the privacy statistics network: create detached intermediate dataset with private labels to train the privacy ReMINE network to converge on I(Z; S(X)), and then calculate an average privacy leakage score across `num_batches_final_MI` iterations.
  - Combine both utility score and privacy leakage as the loss term and take a step updating Infoshape's encoder's weights.

Notes:
- Param for number of InfoShape encoder training epochs (50 was chosen since each epoch takes some amount of time < 1 hr, and we wanted to show InfoShape's potential effectiveness as a POC).
- Supplying the private labels to the dual optimizer class's constructor is necessary for dual optimization. Could probably refactor this to be better.
- Param for batched gradient accumulation to lower the variance of the ReMINE convergence.
- Params for number of ReMINE iterations (experimentally identified numbers that indicated convergence)
- ReMINE regularization params (see ReMINE paper's objective function differences from that of MINE)
- Minibatch size for ReMINE network training
- Params giving options for saving encoder weights and ReMINE convergence graphs in Tensorboard
- Options to exclude either privacy or utility from legacy experiments

In [ ]:
N_INFOSHAPE_EPOCHS = 2
N_PRIVACY_MINE_EPOCHS = 2
N_UTILITY_MINE_EPOCHS = 2
GRADIENT_BATCH_SIZE = 10  # How many gradients get accumulated before update (zero_grad)
EXPERIMENT = f"[DEBUG][SYNTHETIC DATA | PUBLIC PRIVATE LABELS | DUAL ENC TRAIN] REMINE BS=2K C=0 λ=0.1 MIb=10 ENC_EPOCHS={N_INFOSHAPE_EPOCHS}"

In [ ]:
dualopt_model = DualOptimizationDenseEncoder(
    train_loader_pub,
    mine_epochs_privacy=N_PRIVACY_MINE_EPOCHS,
    mine_epochs_utility=N_UTILITY_MINE_EPOCHS,
    enc_out_nodes=N_ENC_OUT_NODES,
    private_labels=y_train_pri
).to(device)
start = time.time()
dualopt_model.train_encoder(
    num_enc_epochs=N_INFOSHAPE_EPOCHS,
    num_batches_final_MI=3,
    include_privacy=True,
    include_utility=True,
    K=MINE_BATCH_SIZE,
    gradient_batch_size=GRADIENT_BATCH_SIZE,
    save_enc_weights=True,
    enc_save_path=ENC_SAVE_PATH
)
print(f"Total time taken: {time.time() - start} s.")

#### Trained Encoder AUC Reevaluation

Key insight is drawn from comparing between untrained and ReMINE-trained encoder

##### Public Labels

In [ ]:
enc_trained = torch.load(ENC_SAVE_PATH + f"{EXPERIMENT} epoch={N_INFOSHAPE_EPOCHS-1}.pt")  # epochs are 0 indexed

train_transform_pub_trainedenc = enc_trained(train_dataset_pub.data.float()).detach()
test_transform_pub_trainedenc = enc_trained(test_dataset_pub.data.float()).detach()
train_data_transform_pub_trainedenc = SyntheticDataset(
    train_transform_pub_trainedenc,
    train_dataset_pub.targets
)
test_data_transform_pub_trainedenc = SyntheticDataset(
    test_transform_pub_trainedenc,
    test_dataset_pub.targets
)

train_loader_transform = DataLoader(train_data_transform_pub_trainedenc, batch_size=BATCH_SIZE, shuffle=True)
test_loader_transform = DataLoader(test_data_transform_pub_trainedenc, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model_enctrans_ = DenseClassifier(enc_trained.out_nodes).to(device)
model_enctrans_.train_classifier(train_loader_transform, epochs=N_CLASSIFIER_TRAINING_EPOCHS)

In [ ]:
model_enctrans_.evaluate(test_loader_transform, experiment_save_file="[refactor2] trainedenc data roc auc pub.json")

##### Private Labels

In [ ]:
# Use the same encoder, dual-objective trained with ReMINE
train_transform_pri_trainedenc = enc_trained(train_dataset_pri.data.float()).detach()
test_transform_pri_trainedenc = enc_trained(test_dataset_pri.data.float()).detach()
train_data_transform_pri_trainedenc = SyntheticDataset(
    train_transform_pri_trainedenc,
    train_dataset_pri.targets
)
test_data_transform_pri_trainedenc = SyntheticDataset(
    test_transform_pri_trainedenc,
    test_dataset_pri.targets
)

train_loader_transform_pri = DataLoader(train_data_transform_pri_trainedenc, batch_size=BATCH_SIZE, shuffle=True)
test_loader_transform_pri = DataLoader(test_data_transform_pri_trainedenc, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model_enctrans_ = DenseClassifier(enc_trained.out_nodes).to(device)
model_enctrans_.train_classifier(train_loader_transform_pri, epochs=N_CLASSIFIER_TRAINING_EPOCHS)

In [ ]:
model_enctrans_.evaluate(test_loader_transform_pri, experiment_save_file="[refactor2] trainedenc data roc auc pri.json")

## Upload ReMINE data to Tensorboard Dev
Use dataframe to then export for making plots in Excel

In [ ]:
!tensorboard dev upload --logdir "lightning_logs/[FINAL50][SYNTHETIC DATA | PUBLIC PRIVATE LABELS | DUAL ENC TRAIN] REMINE BS=2K C=0 λ=0.1 MIb=10 ENC_EPOCHS=10 utility BS=2000" \
  --name "ReMINE FINAL50 [PUBLIC PRIVATE LABELS | DUAL ENC TRAIN] REMINE BS=2K ENC_EPOCHS=50 utility" \
  --one_shot

In [ ]:
experiment_id = "7qGGSyFEQDKvbE7kyahxAg"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
utility_df = experiment.get_scalars()
utility_df

In [ ]:
!tensorboard dev upload --logdir "lightning_logs/[FINAL50][SYNTHETIC DATA | PUBLIC PRIVATE LABELS | DUAL ENC TRAIN] REMINE BS=2K C=0 λ=0.1 MIb=10 ENC_EPOCHS=10 privacy BS=2000" \
  --name "ReMINE FINAL50 [PUBLIC PRIVATE LABELS | DUAL ENC TRAIN] REMINE BS=2K ENC_EPOCHS=50 privacy" \
  --one_shot

In [ ]:
experiment_id = "w0VkQnwwSZ2av6qC5GNn3g"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
privacy_df = experiment.get_scalars()
privacy_df

In [ ]:
import json

with open("data/icassp/privacy/remine_privacy50.json", "w") as f:
    f.write(privacy_df.to_json(orient="records", lines=True))
with open("data/icassp/utility/remine_utility50.json", "w") as g:
    g.write(utility_df.to_json(orient="records", lines=True))